In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as sps
import scipy.optimize


%matplotlib inline

density:
$$f_0(x,a,b) = \frac{3b^3}{(x - a)^4}I(x > a + b)$$, $$f_1(x, a, b) = \frac{x - a}{b^2}e^{-\frac{x - a}{b}}I(x > a)$$

Легко заметить, что  $f_0 - $ немного сдвинутое распределение с параметрами $Pareto(b, 3)$  и сдвинутое на a

А второе - Гамма(1,2)

In [197]:
pareto = sps.pareto(3)
gamma = sps.gamma(2)

In [198]:
a = 3
b = 4

In [202]:
sample = (sps.pareto.rvs(3, loc=a, scale=b, size=int(1e5)))

In [203]:
(sps.pareto.pdf(sample,3, a, b)).mean()

0.32252566638028657

In [204]:
from tqdm import tqdm

In [205]:
# prodik = 0
# for elem in tqdm(sample):
#     prodik += np.log(sps.pareto.pdf(elem, 3, a, b))

In [173]:
# def mle(arg, X=sample):
#     minik = X.min()
#     if(minik <= arg[0] + arg[1]):
#         return -np.inf
#     return -(sum(np.log( sps.pareto.pdf(X,3, arg[0], arg[1]))))

In [206]:
sample.min()

7.0000035622862828

In [225]:
def mle_0(arg, X=sample):
    minik = X.min()
    if(minik <= arg[0] + arg[1]):
        return np.inf
    return -(sum(np.log( sps.pareto.pdf(X,3, arg[0], arg[1]))))

In [224]:
def mle(a, X=sample):
    minek = X.min()
    aksdkasd = 9 * len(X) * (minek - a) - 4 * sum(1 / (X - a))
    return -aksdkasd

In [226]:
opres = scipy.optimize.minimize(mle_0, x0=(1e-8, 2))
opres

      fun: 414263.2238863554
 hess_inv: array([[ 1.12516317,  0.04191411],
       [ 0.04191411,  0.00158089]])
      jac: array([ -49000.91015625, -101233.4140625 ])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 53
      nit: 1
     njev: 12
   status: 2
  success: False
        x: array([ 0.30306857,  2.96345703])

In [212]:
mle(opres.x[0])

-322122553200002.31

Теперь посчитаем для второго

In [183]:
# sps.gamma()

In [184]:
# sample = (sps.gamma.rvs(3, loc=a, scale=b, size=int(1e5)))

In [185]:
# sample

Найдем максимум вручную для Гамма

In [227]:
def hit(arg, X=sample):
    return sum(np.log(sample - arg[0])) -2 * len(sample) * np.log(arg[1]) - sum(sample - arg[0]) / arg[1]

In [228]:
sample

array([ 10.34280885,   7.05541556,   8.29884791, ...,   7.56038708,
         8.15394733,   7.80672251])

Слишком долго! Прибегнем к минимизации через scipy.optimize.minimize

In [229]:
%%time
sps.pareto.fit(sample)

CPU times: user 1min 36s, sys: 923 ms, total: 1min 37s
Wall time: 1min 42s


(2.9080238349384802, 3.1617538832260346, 3.8382496664134651)

In [158]:
%%time
sps.gamma.fit(sample)

CPU times: user 1min 28s, sys: 306 ms, total: 1min 28s
Wall time: 1min 29s


(0.040145983094392865, 5.000011782352483, 1.370373260125477)

In [193]:
min(sample)

3.0000139328311719

In [195]:
scipy.optimize.minimize(hit, x0=(1e-6,3))

      fun: nan
 hess_inv: array([[ 0.00199672,  0.0066093 ],
       [ 0.0066093 ,  0.02263552]])
      jac: array([ nan,  nan])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 460
      nit: 3
     njev: 115
   status: 2
  success: False
        x: array([  6183.42830772,  20937.66035292])

В итоге никак не вышло найти RML. Пытался и аналитически и тут посчитать - не сходится